# YouTube → Download → Gofile Upload

**GitHub repo:** [YT-Download-Gofile-Upload](https://github.com/ahmed98Osama/YT-Download-Gofile-Upload)

**What this notebook does:** Download YouTube videos or playlists with [yt-dlp](https://github.com/yt-dlp/yt-dlp), optionally zip the results, and upload to [Gofile.io](https://gofile.io) for shareable links (No Gofile account required). Supports optional cookies (e.g. for age-restricted content) and an optional URLs file for many links.

 **Section 1:** Install yt-dlp. 

 **Section 2:** one video or playlist → download → upload each file to Gofile. 

 **Section 3:** full workflow — multiple URLs/playlists → optional(cookies) → download → zip → upload zips to Gofile. 

<a href="https://colab.research.google.com/github/ahmed98Osama/YT-Download-Gofile-Upload/blob/main/Download_YouTube_Videos_Using_yt_dlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install yt-dlp

Install or upgrade the `yt-dlp` package (required for downloading).

In [ ]:
%pip install yt-dlp

## 2. Simple single-video download & Upload to Gofile (curl)

Download one YouTube video in best quality (interactive: prompts for URL and save path), then optionally upload a file to Gofile.io via `curl`. Adjust the curl file path to match your downloaded file.

In [ ]:
import os
import yt_dlp  # pyright: ignore[reportMissingImports]

def download_youtube_simple(video_url, save_path="downloads"):
    """
    Downloads one or more YouTube videos (single URL or playlist) in best quality.
    Returns (list of local_paths, list of display_names) for upload; ([], []) on error.

    Args:
        video_url (str): URL of a YouTube video or playlist.
        save_path (str): Directory to save the downloaded video(s).
    """
    downloaded_paths = []  # collect all finished files (supports playlists)
    downloaded_display_names = []

    def _progress_hook(d):
        if d.get('status') == 'finished':
            info = d.get('info_dict', {})
            path = info.get('_filename')
            # When upload_date is missing, yt-dlp uses "NA"; rename to avoid _NA in filename
            if path and '_NA.' in path:
                new_path = path.replace('_NA.', '.')
                try:
                    os.rename(path, new_path)
                    info['_filename'] = new_path
                    path = new_path
                except OSError:
                    pass
            if path:
                downloaded_paths.append(path)
                title = (info.get('title') or 'video').replace('/', '_').replace('\\', '_')
                ext = info.get('ext') or 'mp4'
                downloaded_display_names.append(f"{title}.{ext}")

    try:
        ydl_opts = {
            'format': 'bestvideo*+bestaudio*/bestvideo+bestaudio/bv*+ba*/bv+ba/best',  # Fallbacks when separate streams unavailable (e.g. SABR); /best takes single combined stream
            'outtmpl': f'{save_path}/%(autonumber)03d_%(title)s_%(upload_date)s.%(ext)s',  # Number, title, date (YYYYMMDD)
            'autonumber_start': 1,
            'merge_output_format': 'mp4',  # Merge video and audio into MP4 format
            'progress_hooks': [_progress_hook],
            'ignoreerrors': True,  # Skip failed entries in a playlist
        }
        print("Downloading...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        print(f"Video(s) downloaded successfully and saved in: {save_path}")
        return downloaded_paths, downloaded_display_names
    except Exception as e:
        print(f"An error occurred: {e}")
        return [], []

if __name__ == "__main__":
    # Input YouTube video or playlist URL
    video_url = input("Enter the YouTube video URL: ").strip()

    # Input save directory (default is 'downloads')
    save_path = input("Enter the save path (or press Enter for default 'downloads'): ").strip() or "downloads"

    # Download; returns lists so playlists upload all files
    downloaded_files, file_display_names = download_youtube_simple(video_url, save_path)

    # --- Optional: upload all downloaded files to Gofile.io (filename = video title when available) ---
    if downloaded_files:
        import subprocess
        uploaded_count = 0
        for path, name in zip(downloaded_files, file_display_names):
            try:
                if name:
                    # Sanitize filename so semicolons/quotes don't break curl -F syntax
                    safe_name = name.replace(";", "_").replace("\"", "_").replace(chr(10), "_").replace(chr(13), "_")
                    subprocess.run(
                        ["curl", "-F", f"file=@{path};filename={safe_name}", "https://upload.gofile.io/uploadfile"],
                        check=True
                    )
                else:
                    subprocess.run(
                        ["curl", "-F", f"file=@{path}", "https://upload.gofile.io/uploadfile"],
                        check=True
                    )
                uploaded_count += 1
            except (subprocess.CalledProcessError, FileNotFoundError) as e:
                print(f"Failed to upload {path}: {e}")
        print(f"Uploaded {uploaded_count} of {len(downloaded_files)} file(s) to Gofile.io.")
    else:
        print("Skipping upload (no files were downloaded).")

## 3. Full workflow: Download → Zip → Upload to Gofile

Download YouTube videos or playlists (with optional cookies), zip the results, and upload to Gofile.io. Supports cookie file path or paste (e.g. from Get cookies.txt LOCALLY).

### Add cookies (optional)

Paste your Netscape-format cookies below (e.g. from [Get cookies.txt LOCALLY](https://github.com/kairi003/Get-cookies.txt-Locally)), then run the cell. The file will be saved and used by the full workflow if you set `CONFIG["cookie_file_path"]` to the path shown.

In [ ]:
# Path where the cookie file will be saved (used by the full workflow cell below)
COOKIE_FILE_PATH = "cookies.txt"

# Paste your Netscape-format cookie content between the triple quotes (e.g. export from Get cookies.txt LOCALLY)
COOKIE_CONTENT = """

"""

if COOKIE_CONTENT.strip() and not COOKIE_CONTENT.strip().startswith("# Paste"):
    with open(COOKIE_FILE_PATH, "w", encoding="utf-8", newline="\n") as f:
        f.write(COOKIE_CONTENT.strip())
    print(f"Cookies saved to {COOKIE_FILE_PATH!r}. Set CONFIG['cookie_file_path'] to {COOKIE_FILE_PATH!r} in the full workflow cell below.")
else:
    print("No cookie content to save. Paste cookies between the triple quotes above and run again.")

Cookies saved to 'cookies.txt'. Set CONFIG['cookie_file_path'] to 'cookies.txt' in the full workflow cell below.


### URLs file (optional)

Paste YouTube video or playlist URLs below (one per line). Run the cell to save them to a file. The full workflow will use this file if you set `CONFIG["video_urls_file"]` to the path shown.

In [ ]:
# Path where the URLs file will be saved (used by the full workflow cell below)
URL_FILE_PATH = "urls.txt"

# Paste one URL per line between the triple quotes. Empty lines and lines starting with # are skipped.
URL_CONTENT = """

"""

urls_lines = [line.strip() for line in URL_CONTENT.strip().splitlines() if line.strip() and not line.strip().startswith("#")]
if urls_lines:
    with open(URL_FILE_PATH, "w", encoding="utf-8", newline="\n") as f:
        f.write("\n".join(urls_lines))
    print(f"URLs saved to {URL_FILE_PATH!r} ({len(urls_lines)} URL(s)). Set CONFIG['video_urls_file'] to {URL_FILE_PATH!r} in the full workflow cell below.")
else:
    print("No URLs to save. Paste one URL per line between the triple quotes above and run again.")

### Run the workflow

Run the cell below to download, zip, and upload to Gofile (uses CONFIG and optional cookies/URLs file above).

In [ ]:
import yt_dlp  # pyright: ignore[reportMissingImports]
import os
import re
import shutil
import requests
from typing import Any

# --- Optional config: set USE_CONFIG = True and fill CONFIG to skip interactive prompts ---
USE_CONFIG = True
CONFIG = {
    "video_urls": [

    ],           # Or use video_urls_file to read from a text file
    "video_urls_file": "urls.txt",   # Path to a text file with one URL per line. Keep None unless you run the "URLs file" cell (creates urls.txt) or have the file; otherwise USE_CONFIG will fail with a clear message.
    "base_save_path": "downloads",
    "cookie_file_path": "cookies.txt",   # Path to Netscape cookie file. Set to "cookies.txt" after running the "Add cookies" cell above, or None to skip.
    "compression": "individual",    # "single" or "individual"
}
# Use cookie file from "Add cookies" cell if it exists
try:
    if "COOKIE_FILE_PATH" in dir() and os.path.isfile(COOKIE_FILE_PATH):
        CONFIG["cookie_file_path"] = COOKIE_FILE_PATH
except NameError:
    pass
# Use URLs file from "URLs file" cell if it exists
try:
    if "URL_FILE_PATH" in dir() and os.path.isfile(URL_FILE_PATH):
        CONFIG["video_urls_file"] = URL_FILE_PATH
except NameError:
    pass

def download_youtube_batch(video_urls, base_save_path="downloads", cookie_file=None):
    """
    Downloads YouTube videos or playlists in the best available video and audio quality into dynamically named subfolders.

    Args:
        video_urls (list): A list of URLs for YouTube videos or playlists.
        base_save_path (str): The base directory to save the downloaded content.
        cookie_file (str, optional): Path to a Netscape-format cookie file. Defaults to None.
    """
    # Ensure the base save directory exists
    os.makedirs(base_save_path, exist_ok=True)

    for url in video_urls:
        url = url.strip()
        if not url:
            continue

        try:
            # Common options for yt-dlp, including cookie file if provided
            # Use a permissive format and ignoreerrors so info extraction doesn't fail on partial playlists
            common_ydl_opts: dict[str, Any] = {
                'quiet': True,
                'simulate': True,
                'no_warnings': True,  # Suppress warnings about JS runtime
                'format': 'bestvideo*+bestaudio*/bestvideo+bestaudio/bv*+ba*/bv+ba/best',  # Fallbacks when separate streams unavailable (e.g. SABR); /best takes single combined stream
                'ignoreerrors': True,  # Allow partial playlists when some videos fail (e.g. format unavailable)
            }
            if cookie_file:
                common_ydl_opts['cookiefile'] = cookie_file

            # Initialize a temporary YoutubeDL instance to extract info without downloading
            print(f"Extracting information for URL: {url}")
            with yt_dlp.YoutubeDL(common_ydl_opts) as ydl_info_extractor:
                info_dict = ydl_info_extractor.extract_info(url, download=False)

            if info_dict is None:
                print(f"Could not extract information for {url}. Skipping.")
                continue

            # Determine folder name based on content type (video or playlist)
            if info_dict.get('_type') == 'playlist':
                folder_name = info_dict.get('title', 'Unknown Playlist')
            else:
                folder_name = info_dict.get('title', 'Unknown Video')

            # Sanitize folder name to remove invalid characters, ensuring folder_name is a string
            sanitized_folder_name = re.sub(r'[\\/:*?"<>|]', '_', str(folder_name))
            current_output_path = os.path.join(base_save_path, sanitized_folder_name)

            # Ensure the specific output directory for this video/playlist exists
            os.makedirs(current_output_path, exist_ok=True)
            print(f"Downloading content for '{folder_name}' into: {current_output_path}")

            # When upload_date is missing, yt-dlp uses "NA"; hook renames file to remove _NA
            def _rename_if_na(d):
                if d.get('status') == 'finished':
                    info = d.get('info_dict', {})
                    path = info.get('_filename')
                    if path and '_NA.' in path:
                        new_path = path.replace('_NA.', '.')
                        try:
                            os.rename(path, new_path)
                            info['_filename'] = new_path
                        except OSError:
                            pass
            # Options for yt-dlp for the current download
            # Robust format with fallbacks; ignoreerrors so one bad video doesn't stop a playlist
            ydl_opts_for_current_download: dict[str, Any] = {
                'format': 'bestvideo*+bestaudio*/bestvideo+bestaudio/bv*+ba*/bv+ba/best',  # Fallbacks when separate streams unavailable (e.g. SABR); /best takes single combined stream
                'outtmpl': f'{current_output_path}/%(autonumber)03d_%(title)s_%(upload_date)s.%(ext)s',  # Number, title, date (YYYYMMDD); NA renamed by hook if missing
                'autonumber_start': 1,
                'merge_output_format': 'mp4',  # Merge video and audio into MP4 format
                'no_warnings': True,  # Suppress warnings about JS runtime
                'ignoreerrors': True,  # Skip failed videos in a playlist, continue with others
                'progress_hooks': [_rename_if_na],
            }
            if cookie_file:
                ydl_opts_for_current_download['cookiefile'] = cookie_file

            # Downloading the video/playlist
            with yt_dlp.YoutubeDL(ydl_opts_for_current_download) as ydl_downloader:  # pyright: ignore[reportArgumentType]
                ydl_downloader.download([url])
                print(f"Content for '{folder_name}' downloaded successfully and saved in: {current_output_path}")

        except Exception as e:
            print(f"An error occurred while processing {url}: {e}")

def compress_downloaded_content(base_save_path, compression_choice):
    """
    Compresses downloaded content into zip archives based on user preference.

    Args:
        base_save_path (str): The base directory where content is downloaded.
        compression_choice (str): 'single' for one zip, 'individual' for multiple zips.

    Returns:
        list: A list of paths to the created zip files.
    """
    zip_files = []
    print(f"Starting compression with choice: {compression_choice}")

    if compression_choice == 'single':
        archive_name = os.path.join(base_save_path, os.path.basename(base_save_path) + '_archive')
        try:
            shutil.make_archive(archive_name, 'zip', base_save_path)
            zip_file_path = archive_name + '.zip'
            zip_files.append(zip_file_path)
            print(f"Created single zip archive: {zip_file_path}")
        except Exception as e:
            print(f"Error creating single zip archive: {e}")
    elif compression_choice == 'individual':
        for item in os.listdir(base_save_path):
            item_path = os.path.join(base_save_path, item)
            if os.path.isdir(item_path):
                archive_name = os.path.join(base_save_path, item)
                try:
                    shutil.make_archive(archive_name, 'zip', item_path)
                    zip_file_path = archive_name + '.zip'
                    zip_files.append(zip_file_path)
                    print(f"Created individual zip archive for {item}: {zip_file_path}")
                except Exception as e:
                    print(f"Error creating zip archive for {item}: {e}")
    return zip_files

def upload_to_gofile(file_path):
    """
    Uploads a file to Gofile.io and returns the shareable download URL.

    Args:
        file_path (str): The path to the file to upload.

    Returns:
        str: The shareable download URL, or None if the upload fails.
    """
    try:
        # 1. Get the best available Gofile.io server (try legacy API first, then getServer)
        server_name = None
        try:
            server_response = requests.get("https://api.gofile.io/servers", timeout=15)
            server_response.raise_for_status()
            server_data = server_response.json()
            if server_data.get('status') == 'ok' and server_data.get('data', {}).get('servers'):
                server_name = server_data['data']['servers'][0]['name']
        except (requests.exceptions.RequestException, (KeyError, IndexError, TypeError)):
            pass
        if not server_name:
            try:
                get_server = requests.get("https://gofile.io/api/getServer", timeout=15)
                if get_server.status_code != 200:
                    get_server = requests.get("https://apiv2.gofile.io/getServer", timeout=15)
                get_server.raise_for_status()
                server_data = get_server.json()
                sn = (server_data.get('data') or {}).get('server')
                if sn and (server_data.get('status') != 'error'):
                    server_name = sn
            except (requests.exceptions.RequestException, (KeyError, TypeError)):
                pass
        if not server_name:
            print("Error: Could not get Gofile.io server information. Check https://gofile.io for API status or try again later.")
            return None
        upload_url = f"https://{server_name}.gofile.io/uploadFile"
        print(f"Using Gofile.io server: {server_name}")

        # 2. Upload the file
        with open(file_path, 'rb') as f:
            files = {'file': f}
            upload_response = requests.post(upload_url, files=files)
            upload_response.raise_for_status() # Raise an exception for HTTP errors
            upload_data = upload_response.json()

        if upload_data['status'] == 'ok':
            download_page = upload_data['data']['downloadPage']
            print(f"Successfully uploaded {os.path.basename(file_path)}. Download page: {download_page}")
            return download_page
        else:
            print(f"Gofile.io upload failed for {os.path.basename(file_path)}: {upload_data.get('status', 'Unknown error')}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Network error during Gofile.io upload for {os.path.basename(file_path)}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Gofile.io upload for {os.path.basename(file_path)}: {e}")
        return None

if __name__ == "__main__":
    video_urls = []
    if USE_CONFIG:
        # Load URLs from file if configured (one URL per line; skip empty and # lines)
        urls_file = CONFIG.get("video_urls_file") or None
        if urls_file and os.path.isfile(urls_file):
            with open(urls_file, "r", encoding="utf-8") as f:
                video_urls = [line.strip() for line in f if line.strip() and not line.strip().startswith("#")]
            if not video_urls:
                print(f"URLs file {urls_file!r} is empty or has no valid lines; falling back to CONFIG['video_urls'].")
                video_urls = list(CONFIG.get("video_urls") or [])
        elif urls_file:
            print(f"URLs file {urls_file!r} not found; falling back to CONFIG['video_urls'].")
            video_urls = list(CONFIG.get("video_urls") or [])
            if not video_urls:
                print("USE_CONFIG is True but the URLs file is missing and CONFIG['video_urls'] is empty. Run the \"URLs file\" cell first, or set CONFIG['video_urls'] or CONFIG['video_urls_file'] = None.")
                raise SystemExit(0)
        else:
            video_urls = list(CONFIG.get("video_urls") or [])
    if USE_CONFIG and not video_urls:
        print("USE_CONFIG is True but no URLs were loaded (CONFIG['video_urls'] is empty and CONFIG['video_urls_file'] is unset, missing, or empty). Add URLs or run the \"URLs file\" cell first, or set USE_CONFIG = False for interactive mode.")
        raise SystemExit(0)
    if USE_CONFIG and video_urls:
        base_save_path = CONFIG.get("base_save_path", "downloads") or "downloads"
        cookie_file_path = CONFIG.get("cookie_file_path") or None
        if cookie_file_path:
            if not os.path.isfile(cookie_file_path):
                print(f"Cookie file {cookie_file_path!r} not found; skipping cookies.")
                cookie_file_path = None
            elif os.path.getsize(cookie_file_path) == 0:
                print(f"Cookie file {cookie_file_path!r} is empty; skipping cookies.")
                cookie_file_path = None
        compression_choice = CONFIG.get("compression", "single") or "single"
    else:
        all_urls_input = []
        print("Enter YouTube video or playlist URLs (one per line, press Enter on an empty line to finish):")
        while True:
            url_input = input().strip()
            if not url_input:
                break
            split_urls = re.split(r'[\n,]+', url_input)
            all_urls_input.extend([url.strip() for url in split_urls if url.strip()])
        video_urls = [url for url in all_urls_input if url]
        if not video_urls:
            print("No URLs provided. Exiting.")
            raise SystemExit(0)
        base_save_path = input("Enter the base save path (or press Enter for default 'downloads'): ").strip() or "downloads"
        cookie_file_path = input("Enter path to cookie file (Netscape format, e.g. from Get cookies.txt LOCALLY; press Enter to skip): ").strip() or None
        compression_choice = None
        while compression_choice not in ['single', 'individual']:
            choice_input = input("Create a 'single' (s) zip or 'individual' (i) zips per subfolder? [s/i, default: s]: ").strip().lower()
            if not choice_input or choice_input in ('s', 'single'):
                compression_choice = 'single'
            elif choice_input in ('i', 'individual'):
                compression_choice = 'individual'
            else:
                print("Invalid choice. Enter s (single) or i (individual).")

    print(f"Compression: {compression_choice}")
    download_youtube_batch(video_urls, base_save_path, cookie_file_path)
    compressed_files = compress_downloaded_content(base_save_path, compression_choice)
    print(f"Compressed files: {compressed_files}")
    gofile_links = []
    print("\n--- Uploading files to Gofile.io ---")
    for zip_file in compressed_files:
        link = upload_to_gofile(zip_file)
        if link:
            gofile_links.append(link)
    if gofile_links:
        print("\n--- Gofile.io Download Links ---")
        for i, link in enumerate(gofile_links):
            print(f"Link {i+1}: {link}")
    else:
        print("No files were successfully uploaded to Gofile.io.")